# Creating models with alternative outputs

Models that follow the metatomic interface can have multiple simultaneous outputs. For example, the same model can have an `"energy"` output, as well as an `"energy_uncertainty"` output, or a `"features"` output. 

There is a set of standardized outputs, which is defined in https://docs.metatensor.org/metatomic/latest/outputs/index.html and growing every day! Models can also use non-standardized outputs by naming as `"<prefix>::<output>"`, where `prefix` should indicate the software used to create this output. This allows us to experiment with new outputs outside of metatomic!


In this tutorial, we will define a new version of the model from notebook 2, adding a `"features"` output that will contain the features at the last layer of the neural network.

In [ ]:
from typing import Dict, List, Optional

import ase.io
import numpy as np
import matplotlib.pyplot as plt
import torch

torch.manual_seed(123456)

from featomic.torch import SoapPowerSpectrum
from metatensor.torch import Labels, TensorBlock, TensorMap

import metatomic.torch as mta
import metatensor.torch as mts

from metatomic.torch import System, ModelOutput

In [ ]:
frames = ase.io.read("propenol_conformers_dftb.xyz", ":500")

energies = np.array([[f.info["dftb_energy_eV"]] for f in frames])
forces = np.vstack([f.arrays["dftb_forces_eV_per_Ang"] for f in frames])

To add new outputs to a model, we'll need to add another `TensorMap` to the return `dict` of this model.

The exact outputs the model should produce are determined by whoever is executing the model; and passed in the `outputs: Dict[str, ModelOutput]` parameter. The code inside the model should thus check this structure to determine whether a given output has been requested, as well as whether this ouptut should be expressed as a per-atom or per-structure.

| ![TASK](img/clipboard.png) | Add "features" to the result of the model when `per_atom` is False |
|----------------------------|--------------------------------------------------------------------|

In [ ]:
class SOAPModel(torch.nn.Module):
    def __init__(self, soap_parameters, atomic_types, energy_offset):
        super().__init__()

        self.energy_offset = torch.tensor(energy_offset)
        self.atomic_types = atomic_types

        self.soap_calculator = SoapPowerSpectrum(**soap_parameters)
        self.neighbor_atom_types = Labels(
            ["neighbor_1_type", "neighbor_2_type"], 
            torch.tensor([(i, j) for i in atomic_types for j in atomic_types if i <= j])
        )

        # Number of features produced by the SOAP calculator,
        # i.e. size of the input of the NN
        n_soap = (
            (soap_parameters["basis"]["max_angular"] + 1)
            * (soap_parameters["basis"]["radial"]["max_radial"] + 1) ** 2
            * len(self.neighbor_atom_types)
        )

    
        self.nn = mts.learn.nn.ModuleMap(
            in_keys = Labels("_", torch.tensor([[0]])),
            modules = [torch.nn.Sequential(
                torch.nn.Linear(
                    in_features=n_soap, out_features=128, bias=False, dtype=torch.float64
                ),
                torch.nn.SiLU(),
                torch.nn.Linear(
                    in_features=128, out_features=128, bias=False, dtype=torch.float64
                ),
                torch.nn.SiLU(),
                torch.nn.Linear(
                    in_features=128, out_features=5, bias=False, dtype=torch.float64
                ),
                torch.nn.SiLU(),
            )]
        )

        self.energy_layer = mts.learn.nn.ModuleMap(
            in_keys = Labels("_", torch.tensor([[0]])),
            modules = [torch.nn.Linear(
                in_features=5, out_features=1, bias=True, dtype=torch.float64
            )],
        )

    def forward(
        self,
        systems: List[System],
        outputs: Dict[str, ModelOutput],
        selected_atoms: Optional[Labels] = None,
    ) -> Dict[str, TensorMap]:
        
        soap = self.soap_calculator(systems, selected_samples=selected_atoms)
        soap = soap.keys_to_properties(self.neighbor_atom_types)
        soap = soap.keys_to_samples("center_type")

        nn_features_per_atom = self.nn(soap)
        
        energies_per_atom = self.energy_layer(nn_features_per_atom)
        
        energy = mts.sum_over_samples(energies_per_atom, ["atom", "center_type"])
        energy.block().values[:] += self.energy_offset

        # add the requested outputs to the results
        results: Dict[str, TensorMap] = {}
        if "energy" in outputs:
            results["energy"] = energy

        if "features" in outputs:
            if outputs["features"].per_atom:
                results["features"] = nn_features_per_atom
            else:
                results["features"] =  ...

        return results

We can now train the model as before, although one difference is that in the training loop we now explicitly request the `"energy"` output to be able to create the loss in terms of the energy. 

Notice how we are **not** training the `"features"` output, it is just a derived output that comes from training the model to reproduce the energy of this dataset.

In [ ]:
SOAP_PARAMETERS = {
    "cutoff": {
        "radius": 3.5,
        "smoothing": {
            "type": "ShiftedCosine",
            "width": 0.2
        }
    },
    "density": {
        "type": "Gaussian",
        "width": 0.3
    },
    "basis": {
        "type": "TensorProduct",
        "max_angular": 5,
        "radial": {
            "type": "Gto",
            "max_radial": 5
        }
    }
}

energy_offset = energies.mean()
model = SOAPModel(
    SOAP_PARAMETERS,
    atomic_types=[1, 6, 8],
    energy_offset=energy_offset,
)


In [ ]:
systems = mta.systems_to_torch(frames, dtype=torch.float64)

reference = torch.tensor(energies)
mse_loss = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
epoch = -1

In [ ]:
start = epoch + 1

for epoch in range(start, start + 80):
    optimizer.zero_grad()
    outputs = model(systems, outputs={"energy": ModelOutput(per_atom=False)})

    predicted = outputs["energy"].block().values
    loss = mse_loss(predicted, reference)

    if epoch % 3 == 0:
        print(f"loss at epoch {epoch} is", loss.item())

    loss.backward()
    optimizer.step()

As we saw previously, we can use this model to predict the energy, now explicitly requesting global energies as a model output:

In [ ]:
predicted_energy = model(systems, {"energy": ModelOutput(per_atom=False)})["energy"]

plt.scatter(energies, predicted_energy.block().values.detach().numpy())

x = [np.min(energies), np.max(energies)]
plt.plot(x, x, c="grey")

plt.title("energies")
plt.xlabel("reference / eV")
plt.ylabel("predicted / eV")
plt.show()


| ![TASK](img/clipboard.png) | Run the model and extract the `"features"` output with `per_atom=False` |
|----------------------------|-------------------------------------------------------------------------|

In [ ]:
features = ...

features = features.block().values.detach().numpy()
if features.shape != (len(systems), 5):
    raise Exception("wrong output from the model")

plt.scatter(features[:, 0], features[:, 1])

plt.title("features")
plt.xlabel("NN feature 1")
plt.ylabel("NN feature 2")
plt.show()

# Exporting the model

We can export the model as we did previously, making sure to declare the two possible outputs in the model's capabilities.

One difference from the model in notebook 2 is that we now directly return a `Dict[str, TensorMap]`, so we don't need to define an additional `ExportWrapper` class. In essence, notebook 2 is how it would look like to wrap existing code to make it compatible with metatomic, while this notebook shows an example of directly defining a metatomic model.

In [ ]:
from metatomic.torch import (
    AtomisticModel, 
    System, 
    ModelOutput, 
    ModelMetadata, 
    ModelCapabilities, 
)

In [ ]:
# we now have two outputs:
energy_output = ModelOutput(
    quantity="energy",
    unit="eV",
    per_atom=False,
)

features_output = ModelOutput(
    quantity="",
    per_atom=True,
)

capabilities = ModelCapabilities(
    length_unit="angstrom",
    interaction_range=SOAP_PARAMETERS["cutoff"]["radius"],
    atomic_types=[1, 6, 8],
    supported_devices=["cpu"],
    dtype = "float64",
    # define the two outputs
    outputs={
        "energy": energy_output,
        "features": features_output,
    },
)

metadata = ModelMetadata(
    name="A simple SOAP NN model",
    description="...",
    authors=[],
    references={}
)

In [ ]:
metatensor_model = AtomisticModel(model.eval(), metadata, capabilities)
metatensor_model.save("model-with-features.pt", collect_extensions="extensions")

# Using the `"features"` output

This new output of our model can currently be used by two simulation tools: chemiscope and PLUMED. In PLUMED, the `"features"` output can be used to define custom collective variables for enhanced sampling. See the documentation here for more information: https://docs.metatensor.org/metatomic/latest/engines/plumed.html

In [chemiscope](https://chemiscope.org), the `"features"` output can be used together with `chemiscope.explore` to automatically extract relevant features from a dataset. You can find the corresponding documentation here: https://chemiscope.org/docs/examples/6-explore.html.

In [ ]:
import chemiscope

The first step is to create a `featurizer` function, and chemiscope provides the `metatomic_featurizer` function to automatically convert a metatomic model into a featurizer.

In [ ]:
featurizer = chemiscope.metatomic_featurizer(model="model-with-features.pt", extensions_directory="extensions")

We can then use this featurizer with a dataset (here the full propenol dataset) to see how different structures are seen by the model, and how different structures are mapped to different points in the abstract feature space.

In [ ]:
propenol = ase.io.read("propenol_conformers_dftb.xyz", ":")

chemiscope.explore(propenol, featurizer)

We can also use the same featurizer with a completely different dataset, as long as the atomic types of the dataset are supported by the model. Here for example we can load a dataset of ethanol molecules and visualize the features in the same way!

**HINT:** you can change the features you visualize by clicking on the ☰ icon on the top left!

In [ ]:
ethanol = ase.io.read("ethanol.xyz", ":")
chemiscope.explore(ethanol, featurizer)

Because our model supports `per_atom` features as well, chemiscope can use it to compute the features associated with different atoms in the molecules! The way to do this is to set the `environments` parameter to the list of atom-centered environments of interest. `all_atomic_environments` will set this to all possible atomic environments in the structures.

In [ ]:
chemiscope.explore(ethanol, featurizer, environments=chemiscope.all_atomic_environments(ethanol))